# Permutations of a Simple Circuit

This notebook walks through how to utilize the core semantics of SysML v2 to generate alternative circuits as inputs to an OpenMDAO solution of these circuits. 

## Background

The M1 user model in SysML v2 is meant to be a set of constraints and rules under which legal instances can be created. Those instances should be taken as alternative produced systems and they can be analyzed in that way.

## Libraries Load-Up

Load up PyMBE and its various libraries.

In [ ]:
from pathlib import Path
import networkx as nx
import matplotlib as plt

import pymbe.api as pm

from pymbe.client import SysML2Client
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.interpretation import repack_instance_dictionaries
from pymbe.interpretation.interp_playbooks import (
    build_expression_sequence_templates,
    build_sequence_templates,
    random_generator_playbook,
    random_generator_phase_1_multiplicities,
)
from pymbe.interpretation.results import *
from pymbe.label import get_label_for_id
from pymbe.query.metamodel_navigator import feature_multiplicity
from pymbe.query.query import (
    roll_up_multiplicity,
    roll_up_upper_multiplicity,
    roll_up_multiplicity_for_type,
    get_types_for_feature,
    get_features_typed_by_type,
)
from pymbe.local.stablization import build_stable_id_lookups

## Load Up Model

Read the model from the local JSON file.

In [ ]:
circuit_client = SysML2Client()

circuit_file = Path(pm.__file__).parent / "../../tests/fixtures/Circuit Builder.json"

circuit_client._load_from_file(circuit_file)

circuit_lpg = SysML2LabeledPropertyGraph()
circuit_lpg.model = circuit_client.model

circuit_model = circuit_lpg.model

[id_to_circuit_name_lookup, circuit_name_to_id_lookup] = build_stable_id_lookups(circuit_lpg)

circuit_lpg.model.MAX_MULTIPLICITY = 10

## Explore Contents of Model with M1 in Memory

Use the M1 memory objects to see what is in the current model, starting with the main packages.

In [ ]:
circuit_model.packages

In [ ]:
circuit_model.ownedElement["Circuit Builder"].ownedElement

In [ ]:
circuit_def = circuit_model.ownedElement["Circuit Builder"].ownedElement["Circuit"]

### Circuit and its Features

Here is the circuit and its features, both parts and used connections.

In [ ]:
circuit_def.relationships

In [ ]:
circuit_def.ownedMember

In [ ]:
circuit_def.ownedMember[7].name

In [ ]:
circuit_def.ownedMember[7].multiplicity.lowerBound

In [ ]:
circuit_def.ownedMember[7].multiplicity.upperBound

## Generate M0 instances from the M1 model

Use the M1 model to start creating a series of instances to represent the circuits that should be analyzed.

In [ ]:
m0_interpretations = [
    random_generator_playbook(
        lpg=circuit_lpg,
        name_hints={},
        filtered_feat_packages=[circuit_lpg.model.ownedElement["Circuit Builder"]],
        phase_limit=10
    ) for run in range(0,100)
]

In [ ]:
pprint_interpretation(m0_interpretations[0], circuit_model, False)

## Filter M0 Instances for Reasonable Circuits

Until we get more sophisticated and can interpret constraints, the initial approach is to filter out solutions with unanalyzable layouts or trim the layouts to something more tractable.

### Connector End Checks

Look at the ends of the three main kinds of connectors.

In [ ]:
n2n = circuit_def.ownedMember["Node to Node"]

In [ ]:
p2p = circuit_def.ownedMember["Part to Part"]

In [ ]:
p2n = circuit_def.ownedMember["Part to Node"]

In [ ]:
circuit_def.ownedMember["Part to Part"].endFeature[0]._id

In [ ]:
m0_interpretations[10][p2n.endFeature[0]._id]

In [ ]:
m0_interpretations[10][p2n.endFeature[1]._id]

In [ ]:
m0_interpretations[10][p2p.endFeature[0]._id]

In [ ]:
m0_interpretations[10][p2p.endFeature[1]._id]

In [ ]:
m0_interpretations[10][n2n.endFeature[0]._id]

In [ ]:
m0_interpretations[10][n2n.endFeature[1]._id]